In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pmdarima import auto_arima
import warnings
warnings.filterwarnings('ignore')



/Users/sarabdeepsingh/anaconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [16]:
# Load the data
data = pd.read_csv('retail.csv')
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data = data.set_index('InvoiceDate')

In [17]:
# Let's visualize the data
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['Sales'])
plt.title('Daily Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.grid(True)
plt.tight_layout()
plt.savefig('sales_time_series.png')
plt.close()

In [18]:
# Check for missing values and handle outliers
print("Missing values:", data.isnull().sum())

Missing values: Unnamed: 0    0
Sales         0
dtype: int64


In [19]:
# Detect and handle outliers
Q1 = data['Sales'].quantile(0.25)
Q3 = data['Sales'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = data[(data['Sales'] < lower_bound) | (data['Sales'] > upper_bound)]
print(f"Number of outliers: {len(outliers)}")

Number of outliers: 21


In [20]:
# For extreme outliers, we can cap them but still preserve the trend
data_processed = data.copy()
data_processed.loc[data_processed['Sales'] > upper_bound * 2, 'Sales'] = upper_bound * 2


In [21]:

# Plot the processed data
plt.figure(figsize=(12, 6))
plt.plot(data_processed.index, data_processed['Sales'])
plt.title('Daily Sales (Processed)')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.grid(True)
plt.tight_layout()
plt.savefig('sales_processed.png')
plt.close()

In [22]:

# Resample to weekly data to reduce noise (optional)
weekly_data = data_processed.resample('W').mean()

# Split the data: use 80% for training, 20% for testing
train_size = int(len(data_processed) * 0.8)
train_data = data_processed.iloc[:train_size]
test_data = data_processed.iloc[train_size:]

print(f"Training data: {train_data.index.min()} to {train_data.index.max()}")
print(f"Testing data: {test_data.index.min()} to {test_data.index.max()}")

Training data: 2010-12-01 00:00:00 to 2011-09-25 00:00:00
Testing data: 2011-09-26 00:00:00 to 2011-12-09 00:00:00


In [23]:
# Create a function to evaluate models
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print(f"{model_name} - MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")
    return {'mae': mae, 'rmse': rmse, 'mape': mape}

In [24]:

# 1. SARIMA Model
# Use auto_arima to find the best parameters
sarima_model = auto_arima(
    train_data['Sales'],
    seasonal=True, 
    m=7,  # Weekly seasonality
    start_p=0, start_q=0,
    max_p=3, max_q=3,
    start_P=0, start_Q=0,
    max_P=2, max_Q=2,
    d=1, D=1,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

print("Best SARIMA parameters:", sarima_model.order, sarima_model.seasonal_order)


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=2332.100, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=2250.405, Time=0.06 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=0.24 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=2302.557, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=2238.527, Time=0.13 sec
 ARIMA(1,1,0)(2,1,1)[7]             : AIC=inf, Time=0.36 sec
 ARIMA(1,1,0)(1,1,1)[7]             : AIC=inf, Time=0.29 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=2275.137, Time=0.10 sec
 ARIMA(2,1,0)(2,1,0)[7]             : AIC=2194.419, Time=0.19 sec
 ARIMA(2,1,0)(1,1,0)[7]             : AIC=2200.472, Time=0.12 sec
 ARIMA(2,1,0)(2,1,1)[7]             : AIC=inf, Time=0.83 sec
 ARIMA(2,1,0)(1,1,1)[7]             : AIC=inf, Time=0.23 sec
 ARIMA(3,1,0)(2,1,0)[7]             : AIC=2158.138, Time=0.23 sec
 ARIMA(3,1,0)(1,1,0)[7]             : AIC=2169.271, Time=0.15 sec
 ARIMA(3,1,0)(2,1,1)[7]             : AIC=inf, Time=0.84 s

In [25]:
# Fit the model
final_sarima = SARIMAX(
    train_data['Sales'],
    order=sarima_model.order,
    seasonal_order=sarima_model.seasonal_order
)
sarima_fitted = final_sarima.fit(disp=False)

In [26]:
# Forecast
sarima_forecast = sarima_fitted.forecast(steps=len(test_data))
sarima_metrics = evaluate_model(test_data['Sales'], sarima_forecast, "SARIMA")

SARIMA - MAE: 10.16, RMSE: 14.60, MAPE: 48.92%


In [28]:
# 2. Prophet Model
# Prepare data for Prophet
prophet_data = data_processed.reset_index()
# Check the actual columns in your data
print("Columns in data:", prophet_data.columns)

# If there are 3 columns, we need to select just the 2 we need
# Assuming the first column is the index/date and one of the others is 'Sales'
prophet_data = prophet_data[['InvoiceDate', 'Sales']]
prophet_data.columns = ['ds', 'y']

# Fit Prophet model
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=0.05
)
prophet_model.fit(prophet_data.iloc[:train_size])

Columns in data: Index(['InvoiceDate', 'Unnamed: 0', 'Sales'], dtype='object')


17:55:17 - cmdstanpy - INFO - Chain [1] start processing
17:55:18 - cmdstanpy - INFO - Chain [1] done processing


In [29]:

# Forecast
future = prophet_model.make_future_dataframe(periods=len(test_data), freq='D')
prophet_forecast = prophet_model.predict(future)


In [30]:
# Evaluate on test data
prophet_pred = prophet_forecast.iloc[train_size:]['yhat'].values
prophet_metrics = evaluate_model(test_data['Sales'].values, prophet_pred, "Prophet")


Prophet - MAE: 6.30, RMSE: 9.49, MAPE: 30.03%


In [31]:
# Plot Prophet components
prophet_fig = prophet_model.plot_components(prophet_forecast)
plt.savefig('prophet_components.png')
plt.close()

In [32]:
# 3. Exponential Smoothing (ETS)
ets_model = ExponentialSmoothing(
    train_data['Sales'],
    seasonal='add',
    seasonal_periods=7,  # Weekly seasonality
    trend='add'
)
ets_fitted = ets_model.fit()

In [33]:
# Forecast
ets_forecast = ets_fitted.forecast(steps=len(test_data))
ets_metrics = evaluate_model(test_data['Sales'], ets_forecast, "ETS")

# Compare models and select the best one
models = {
    'SARIMA': {'metrics': sarima_metrics, 'forecast': sarima_forecast},
    'Prophet': {'metrics': prophet_metrics, 'forecast': prophet_pred},
    'ETS': {'metrics': ets_metrics, 'forecast': ets_forecast}
}

ETS - MAE: 6.08, RMSE: 8.54, MAPE: 32.32%


In [34]:
# Find the best model based on RMSE
best_model = min(models.items(), key=lambda x: x[1]['metrics']['rmse'])[0]
print(f"\nBest model based on RMSE: {best_model}")


Best model based on RMSE: ETS


In [35]:
# Plot actual vs. forecasted values for the best model
plt.figure(figsize=(12, 6))
plt.plot(data_processed.index, data_processed['Sales'], label='Actual')
plt.plot(test_data.index, models[best_model]['forecast'], 'r--', label=f'{best_model} Forecast')
plt.axvline(x=train_data.index[-1], color='k', linestyle='--')
plt.title(f'Sales Forecast using {best_model}')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('best_model_forecast.png')
plt.close()

In [36]:
# Now forecast future sales (next 30 days)
if best_model == 'SARIMA':
    future_model = SARIMAX(
        data_processed['Sales'],
        order=sarima_model.order,
        seasonal_order=sarima_model.seasonal_order
    )
    future_fitted = future_model.fit(disp=False)
    future_forecast = future_fitted.forecast(steps=30)
    future_index = pd.date_range(start=data_processed.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
    
elif best_model == 'Prophet':
    future = prophet_model.make_future_dataframe(periods=len(test_data) + 30, freq='D')
    prophet_future = prophet_model.predict(future)
    future_forecast = prophet_future.iloc[-30:]['yhat'].values
    future_index = pd.date_range(start=data_processed.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
    
else:  # ETS
    ets_full_model = ExponentialSmoothing(
        data_processed['Sales'],
        seasonal='add',
        seasonal_periods=7,
        trend='add'
    )
    ets_full_fitted = ets_full_model.fit()
    future_forecast = ets_full_fitted.forecast(steps=30)
    future_index = pd.date_range(start=data_processed.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')

In [37]:

# Create a dataframe with future predictions
future_df = pd.DataFrame({
    'Date': future_index,
    'Forecasted_Sales': future_forecast
})

print("\nFuture Sales Forecast (Next 30 days):")
print(future_df.head(10))

# Plot the historical data and future forecast
plt.figure(figsize=(15, 7))
plt.plot(data_processed.index, data_processed['Sales'], label='Historical Sales')
plt.plot(future_index, future_forecast, 'r--', label='Future Forecast')
plt.title(f'Sales Forecast for Next 30 Days using {best_model}')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('future_forecast.png')
plt.close()


Future Sales Forecast (Next 30 days):
                 Date  Forecasted_Sales
2011-12-10 2011-12-10         18.053176
2011-12-11 2011-12-11         12.141784
2011-12-12 2011-12-12         20.308052
2011-12-13 2011-12-13         25.749542
2011-12-14 2011-12-14         22.132610
2011-12-15 2011-12-15         24.117335
2011-12-16 2011-12-16         24.505457
2011-12-17 2011-12-17         18.024358
2011-12-18 2011-12-18         12.112966
2011-12-19 2011-12-19         20.279234


In [38]:
# Feature importance analysis (for Prophet)
if best_model == 'Prophet':
    # Extract the components and their contribution
    components = prophet_forecast[['trend', 'weekly', 'yearly']]
    
    # Calculate the magnitude of each component
    component_influence = {
        'Trend': components['trend'].max() - components['trend'].min(),
        'Weekly Seasonality': components['weekly'].max() - components['weekly'].min(),
        'Yearly Seasonality': components['yearly'].max() - components['yearly'].min()
    }
    
    print("\nComponent Influence in Sales Patterns:")
    for component, influence in component_influence.items():
        print(f"{component}: {influence:.2f}")
    
    # Plot the components
    plt.figure(figsize=(12, 8))
    plt.subplot(3, 1, 1)
    plt.plot(prophet_forecast['ds'], prophet_forecast['trend'])
    plt.title('Trend Component')
    plt.grid(True)
    
    plt.subplot(3, 1, 2)
    plt.plot(prophet_forecast['ds'], prophet_forecast['weekly'])
    plt.title('Weekly Seasonality Component')
    plt.grid(True)
    
    plt.subplot(3, 1, 3)
    plt.plot(prophet_forecast['ds'], prophet_forecast['yearly'])
    plt.title('Yearly Seasonality Component')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('component_analysis.png')
    plt.close()



In [39]:
# Save the model and forecasts
import pickle

if best_model == 'SARIMA':
    model_to_save = sarima_fitted
elif best_model == 'Prophet':
    model_to_save = prophet_model
else:
    model_to_save = ets_full_fitted

# Save the final model
with open(f'{best_model}_model.pkl', 'wb') as f:
    pickle.dump(model_to_save, f)

# Save the future forecasts
future_df.to_csv('future_sales_forecast.csv', index=False)

print(f"\n{best_model} model and future forecasts saved successfully!")


ETS model and future forecasts saved successfully!
